# Executor

In [1]:
%run -m literary.notebook

In [1]:
import asyncio
from typing import List

In [ ]:
class Executor:
    def map(self, func, /, *iterables) -> List[asyncio.Future]:
        return [self.submit(func, *args) for args in zip(*iterables)]

    def submit(self, func, /, *args, **kwargs) -> asyncio.Future:
        raise NotImplementedError

    async def retrieve(self, handle: asyncio.Future):
        raise NotImplementedError